In [1]:
import pandas as pd

In [2]:
ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv") 
display(ratings_df.head(10))

C:\Users\sofia\AppData\Local\Temp\ipykernel_2976\2223713249.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv")


,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38
5,876,1211463,111395,1955,5.0,2021-12-02 23:12:49
6,1005,1076348,111433,1955,4.5,2021-06-19 19:53:56
7,1020,1147051,111429,1955,5.0,2018-07-08 20:09:46
8,1029,1225931,111431,1955,5.0,2017-04-24 01:41:52
9,1399,1197513,111415,1958,5.0,2014-07-04 01:07:16


In [3]:
wines_df = pd.read_csv("Dataset\last\XWines_Slim_1K_wines.csv", index_col="WineID")
wines_df['WineID'] = wines_df.index
display(wines_df.head(3))

,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [4]:

body_mapping = {
    'Very light-bodied': 1,
    'Light-bodied': 2,
    'Medium-bodied': 3,
    'Full-bodied': 4,
    'Very full-bodied': 5
}
acidity_mapping = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

type_mapping = {
    'White': 1,
    'Rosé': 2,
    'Red': 3,
    'Sparkling': 4,
    'Dessert': 5,
    'Dessert/Port': 6
}



# Apply the mappings to the DataFrame
wines_df['Body'] = wines_df['Body'].map(body_mapping)
wines_df['Acidity'] = wines_df['Acidity'].map(acidity_mapping)
wines_df['Type'] = wines_df['Type'].map(type_mapping)
display(wines_df.head(3))


,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,4,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,3,3,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,3,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,3,2,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,3,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,4,3,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import scipy.sparse as sp
import numpy as np


wine_features = wines_df[['Type', 'Grapes', 'Harmonize', 'ABV', 'Body', 'Acidity', 'Vintages']]

tfidf = TfidfVectorizer(stop_words='english')
wine_tfidf = tfidf.fit_transform(wine_features['Harmonize'].fillna(''))

# Label Encoding 
label_encoder = LabelEncoder()

type_encoded = label_encoder.fit_transform(wine_features['Type'])
body_encoded = label_encoder.fit_transform(wine_features['Body'])
acidity_encoded = label_encoder.fit_transform(wine_features['Acidity'])
abv_encoded = label_encoder.fit_transform(wine_features['ABV'])


wine_label_encoding = np.column_stack([type_encoded, body_encoded, acidity_encoded, abv_encoded])


wine_features_combined = sp.hstack([wine_tfidf, wine_label_encoding])


cosine_sim = cosine_similarity(wine_features_combined, wine_features_combined)


In [6]:
#FOR WINE NAME

wines_df = wines_df.reset_index(drop=True)

def get_recommendations(wine_name, cosine_sim=cosine_sim):
    if wine_name not in wines_df['WineName'].values:
        return f"Wine '{wine_name}' not found. Please check the wine name."

    idx = wines_df[wines_df['WineName'] == wine_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:6]
    wine_indices = [i[0] for i in sim_scores]
    wine_similarities = [i[1] for i in sim_scores]
    
    recommendations = wines_df.iloc[wine_indices][['WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
    recommendations['Similarity'] = wine_similarities
    
    return recommendations



In [98]:

wines_df = wines_df.reset_index(drop=True)

def get_similarities(wine_id, cosine_sim=cosine_sim):
    if wine_id not in wines_df['WineID'].values:
        return f"Wine with ID '{wine_id}' not found. Please check the WineID."

    idx = wines_df[wines_df['WineID'] == wine_id].index[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:1001]  
    wine_indices = [i[0] for i in sim_scores]
    wine_similarities = [i[1] for i in sim_scores]
    
    similarities = wines_df.iloc[wine_indices][['WineName', 'WineID', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
    
    similarities['Similarity'] = wine_similarities
    
    return similarities


In [91]:
display(get_recommendations("Espumante Moscatel").head(10))
#print(get_recommendations("Ancellotta"))

,WineName,Type,Grapes,Body,Acidity,ABV,Harmonize,Similarity
0,Espumante Moscatel,4,['Muscat/Moscato'],3,3,7.5,"['Pork', 'Rich Fish', 'Shellfish']",1.0
4,Do Lugar Moscatel Espumantes,4,['Muscat/Moscato Bianco'],3,3,7.5,"['Pork', 'Rich Fish', 'Shellfish']",1.0
28,Moscatel,4,['Muscat/Moscato'],3,3,7.5,"['Pork', 'Rich Fish', 'Shellfish']",1.0
33,Terranova Espumante Moscatel,4,['Muscat/Moscato'],3,3,7.5,"['Pork', 'Rich Fish', 'Shellfish']",1.0
15,Malbec Brut Rosé,4,['Malbec'],3,3,11.5,"['Pork', 'Rich Fish', 'Shellfish']",0.8
16,Espumante Método Charmat Brut,4,"['Chardonnay', 'Riesling']",3,3,12.0,"['Pork', 'Rich Fish', 'Shellfish']",0.8


In [132]:
display(get_similarities(100005).head(1001))


,WineName,WineID,Type,Grapes,Body,Acidity,ABV,Harmonize,Similarity
3,Maison de Ville Cabernet-Merlot,100005,3,"['Cabernet Sauvignon', 'Merlot']",4,2,11.0,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
94,Calda Bordaleza Tinto,103070,3,"['Cabernet Sauvignon', 'Merlot', 'Petit Verdot']",4,2,14.0,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",0.8
96,Regional Tinto,103433,3,"['Aragonez', 'Syrah/Shiraz']",4,2,13.0,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",0.8
120,Terras de Basto Tinto,105585,3,"['Azal Tinto', 'Vinhão', 'Padeiro']",4,2,10.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",0.8
208,Saint-Émilion Grand Cru (Premier Grand Cru Cla...,111475,3,['Cabernet Franc'],4,2,13.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",0.8
...,...,...,...,...,...,...,...,...,...
942,Tokaji Aszú 3 Puttonyos,194769,5,['Furmint'],3,3,10.0,"['Sweet Dessert', 'Blue Cheese']",0.0
943,Santorini,194776,1,"['Assyrtiko', 'Aidani', 'Athiri', 'Gaidouria',...",3,3,13.2,"['Shellfish', 'Lean Fish']",0.0
955,Cuvée Alexander Rose de Pinot Extra Brut,195476,4,"['Pinot Noir', 'Cabernet Sauvignon']",3,3,12.5,"['Appetizer', 'Snack', 'Shellfish', 'Rich Fish']",0.0
958,Úrágya Tokaji Furmint,195607,1,['Furmint'],3,3,14.0,"['Shellfish', 'Seafood', 'Salad', 'Maturated C...",0.0


In [211]:
random_user = np.random.choice(ratings_df['UserID'].unique())

user_ratings = ratings_df[ratings_df['UserID'] == random_user]
highest_rated_wine = user_ratings.loc[user_ratings['Rating'].idxmax()]

def get_wine_name(wine_id):
    wine_row = wines_df[wines_df['WineID'] == wine_id]
    if not wine_row.empty:
        return wine_row['WineName'].values[0]
    else:
        return f"Wine ID {wine_id} not found."


print(f"Random User ID: {random_user}")
print(f"Highest Rated Wine for User {random_user}:")
print(f"WineID: {highest_rated_wine['WineID']}, Rating: {highest_rated_wine['Rating']}")


wine_name = get_wine_name(highest_rated_wine['WineID'])
print(f"Wine Name: {wine_name}")
display(get_recommendations(wine_name).head(10))

Random User ID: 1222895
Highest Rated Wine for User 1222895:
WineID: 111417, Rating: 5.0
Wine Name: Pauillac (Grand Cru Classé)


,WineName,Type,Grapes,Body,Acidity,ABV,Harmonize,Similarity
190,Pauillac (Grand Cru Classé),3,"['Cabernet Sauvignon', 'Merlot', 'Cabernet Fra...",4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
517,Camins del Priorat,3,['Garnacha'],4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
523,Priorat,3,"['Garnacha', 'Cabernet Sauvignon', 'Syrah/Shir...",4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
559,Cabernet Sauvignon,3,['Cabernet Sauvignon'],4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
567,Cabernet Sauvignon,3,['Cabernet Sauvignon'],4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0
649,Cocodrilo Red Blend,3,"['Cabernet Sauvignon', 'Malbec', 'Petit Verdot...",4,3,14.5,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",1.0


In [119]:
user_rating_counts = ratings_df.groupby('UserID')['RatingID'].count()

most_ratings_user = user_rating_counts.idxmax()  # UserID with the most ratings
most_ratings_count = user_rating_counts.max()    # Number of ratings for that user

print(f"User {most_ratings_user} has the most ratings with {most_ratings_count} ratings.")

User 1006657 has the most ratings with 67 ratings.


In [131]:
user_ratings = ratings_df[ratings_df['UserID'] == 1006657].sort_values(by='WineID', ascending=True)
user_ratings_high = user_ratings[user_ratings['Rating'] >= 4.5]

high_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_high['WineID'])]

display(high_rated_wines[['WineName', 'WineID']])


,WineName,WineID
202,Châteauneuf-du-Pape,111457
212,Pauillac (Grand Cru Classé),111495
215,Grand Vintage Brut Champagne,111521
230,Emilien,111687
232,Corton-Charlemagne Grand Cru,111713
277,Pomerol,112511
316,Listrac-Médoc,113193
523,Priorat,155438


In [ ]:
wine_name = 'Châteauneuf-du-Pape'

idx = wines_df[wines_df['WineName'] == wine_name].index[0]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

wine_indices = [i[0] for i in sim_scores]
wine_similarities = [i[1] for i in sim_scores]

recommendations = wines_df.iloc[wine_indices][['WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
recommendations['Similarity'] = wine_similarities
display(recommendations.head(20))



In [141]:
wine_to_find = 'Listrac-Médoc'
if wine_to_find in recommendations['WineName'].values:
    
    wine_row = recommendations[recommendations['WineName'] == wine_to_find]
    
    similarity_score = wine_row['Similarity'].values[0]
    
    print(f"The similarity score for {wine_to_find} is: {similarity_score}")
else:
    print(f"{wine_to_find} is not in the recommendations list.")

The similarity score for Listrac-Médoc is: 0.9963638377827774


TO DO:

-for a user, get all top rated wines -done

-for all top wines of user, find similarities. -done

-sum up the wine similarities for all reccommended wines, normalize the score, get the most similar ones overall -done


Evaluation?? idk

-base case: evaluation using only 1 top and 1 bottom wine by 1 user, compare them to the rest of highly liked/disliked wines, detect if they are similar or not to them, as TP,TN,FP,FN - done

-make it work for all the top and bottom wines in a loop, not just one -done

-make it work for the averaged scores of all the high rated wines (the method at the bottom) -done

-make it work for multiple users so we can actually evaluate the model overall -done

-multiple users for the averages -todo

GETTING RANDOM USER WITH >40 RATINGS FOR EVALUATION. GETTING THEIR TOP AND WORST RATED WINES


In [136]:
users_ratings = ratings_df.groupby('UserID').count()

selected = users_ratings['Rating'] > 40
selected_users = users_ratings.loc[selected]

if selected_users.empty:
    print("No users have rated more than 200 items.")
else:
    random_selected = selected_users.sample().index[0]
    print("Selected user: " + str(random_selected))

user_ratings = ratings_df[ratings_df['UserID'] == random_selected].sort_values(by='Rating', ascending=False)
user_ratings_high = user_ratings[user_ratings['Rating'] >=4.5]
user_ratings_low = user_ratings[user_ratings['Rating'] < 3]

high_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_high['WineID'])]
low_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_low['WineID'])]

display(high_rated_wines[['WineName', 'WineID']].head(10))
display(low_rated_wines[['WineName', 'WineID']].head(10))
  

Selected user: 1217395


,WineName,WineID
190,Pauillac (Grand Cru Classé),111417
193,Pauillac (Grand Cru Classé),111422
195,Saint-Julien (Grand Cru Classé),111431
203,Saint-Estèphe (Grand Cru Classé),111458
209,Sauternes (Premier Grand Cru Classé),111478
400,Brunello di Montalcino,137224
717,Grange,174184
786,Dominus (Christian Moueix),179131
797,Cabernet Sauvignon,179855


,WineName,WineID
198,Bourgogne Couvent des Jacobins,111445
238,Bordeaux Rouge,111765
579,Sauvignon Blanc,162912


BASIC EVALUATION: just evaluating if the similarities for 1 top rated wine and 1 bottom rated wine match with the rest of the high/low rated wine


In [137]:
#TRying evaluation with just 1 top rated wine and 1 low rated

fp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, when it shouldn't be similar
fn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, when it should be similar
tp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, and it should be similar
tn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, and it shouldn't be similar
top_wine = high_rated_wines['WineID'].values[0]
similarities = get_similarities(top_wine)  
for wine_id in high_rated_wines['WineID']: 
    if top_wine == wine_id:
        continue   
    if wine_id in (similarities['WineID'].values):
        wine_row = similarities[similarities['WineID'] == wine_id]
        similarity_score = wine_row['Similarity'].values[0]
        if similarity_score >= 0.8:
            tp+=1
        if similarity_score <=0.6:
            fn+=1
for wine_id in low_rated_wines['WineID']:  
    if wine_id in (similarities['WineID'].values): 
        wine_row = similarities[similarities['WineID'] == wine_id]
        similarity_score = wine_row['Similarity'].values[0]
        if similarity_score <=0.6:
            tn+=1
        if similarity_score >= 0.8:
            fp+=1


low_wine = low_rated_wines['WineID'].values[0]
similarities = get_similarities(low_wine) 
for wine_id in low_rated_wines['WineID']: 
    if top_wine == wine_id:
        continue  
    if wine_id in (similarities['WineID'].values):
        wine_row = similarities[similarities['WineID'] == wine_id]
        similarity_score = wine_row['Similarity'].values[0]
        if similarity_score >= 0.8:
            tn+=1
        if similarity_score <=0.6:
            fp+=1
for wine_id in high_rated_wines['WineID']:   
    if wine_id in (similarities['WineID'].values):
        wine_row = similarities[similarities['WineID'] == wine_id]
        similarity_score = wine_row['Similarity'].values[0]
        if similarity_score <=0.6:
            tp+=1
        if similarity_score >= 0.8:
            fn+=1

      
print('tp = '+str(tp))
print('tn = '+str(tn))
print('fp = '+str(fp))
print('fn = '+str(fn))


tp = 9
tn = 3
fp = 2
fn = 3


BASIC BUT BETTER EVALUATION: same thing as before but iterating through all the top/bottom rated wines, not just 1 each

In [140]:
#TRying evaluation with all top rated wine and 1ìall low rated

fp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, when it shouldn't be similar
fn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, when it should be similar
tp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, and it should be similar
tn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, and it shouldn't be similar
for top_wine in high_rated_wines['WineID'].values:
    similarities = get_similarities(top_wine)  
    for wine_id in high_rated_wines['WineID']: 
        if top_wine == wine_id:
            continue   
        if wine_id in (similarities['WineID'].values):
            wine_row = similarities[similarities['WineID'] == wine_id]
            similarity_score = wine_row['Similarity'].values[0]
            if similarity_score >= 0.8:
                tp+=1
            if similarity_score <=0.6:
                fn+=1
    for wine_id in low_rated_wines['WineID']:  
        if wine_id in (similarities['WineID'].values): 
            wine_row = similarities[similarities['WineID'] == wine_id]
            similarity_score = wine_row['Similarity'].values[0]
            if similarity_score <=0.6:
                tn+=1
            if similarity_score >= 0.8:
                fp+=1


for low_wine in low_rated_wines['WineID'].values:
    similarities = get_similarities(low_wine)
    for wine_id in low_rated_wines['WineID']: 
        if top_wine == wine_id:
            continue  
        if wine_id in (similarities['WineID'].values):
            wine_row = similarities[similarities['WineID'] == wine_id]
            similarity_score = wine_row['Similarity'].values[0]
            if similarity_score >= 0.8:
                tn+=1
            if similarity_score <=0.6:
                fp+=1
    for wine_id in high_rated_wines['WineID']:   
        if wine_id in (similarities['WineID'].values):
            wine_row = similarities[similarities['WineID'] == wine_id]
            similarity_score = wine_row['Similarity'].values[0]
            if similarity_score <=0.6:
                tp+=1
            if similarity_score >= 0.8:
                fn+=1

      
print('tp = '+str(tp))
print('tn = '+str(tn))
print('fp = '+str(fp))
print('fn = '+str(fn))


tp = 23
tn = 26
fp = 6
fn = 54


AVERAGED EVALUATION: for 1 user, we first compute the averaged similarities (look at code below this section) that we get from the top rated wines. Then we check if these match with the top and bottom rated wines. We could do the same for the bottom averaged similarities, but I still have to do the method


In [142]:
#RUN CODE BELOW FIRST TO GET similarity_df !!!!!

fp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, when it shouldn't be similar
fn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, when it should be similar
tp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, and it should be similar
tn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, and it shouldn't be similar


similarities = similarity_df
for top_wine in high_rated_wines['WineID'].values:
    if top_wine in (similarities['WineID'].values):
        wine_row = similarities[similarities['WineID'] == top_wine]
        similarity_score = wine_row['Normalized_Similarity'].values[0]
        print(similarity_score)
        if similarity_score >= 0.6:
            tp+=1
        else:
            fn+=1

for low_wine in low_rated_wines['WineID'].values:
    if low_wine in (similarities['WineID'].values):
        wine_row = similarities[similarities['WineID'] == low_wine]
        similarity_score = wine_row['Normalized_Similarity'].values[0]
        print(similarity_score)
        if similarity_score <=0.4:
            tn+=1
        else:
            fp+=1

print('tp = '+str(tp))
print('tn = '+str(tn))
print('fp = '+str(fp))
print('fn = '+str(fn))

0.700692184174146
0.6562477397297015
0.6562477397297015
0.6562477397297015
0.38175021005606574
0.5895810730630351
0.6340255175074794
0.5895810730630351
0.4421510548306446
0.4507272601043167
0.5840605934376499
0.1855192735655726
tp = 5
tn = 1
fp = 2
fn = 4


CODE TO GET AVERAGED SIMILARITIES FROM THE TOP RATED WINES OF THE USER

In [141]:


similarity_df = pd.DataFrame({
    'Wineid': wines_df['WineID'],
    'WineName': wines_df['WineName'],  
    'Accumulated_Similarity': 0.0
})
similarity_df.set_index('Wineid', inplace=True)
similarity_df['WineID'] = similarity_df.index


for wine_to_grade in similarity_df['WineID']:   
    
    tempScore = 0
    for wine_id in high_rated_wines['WineID']:
        
        similarities = get_similarities(wine_id)  
        
           
        if wine_to_grade in (similarities['WineID'].values):
            wine_row = similarities[similarities['WineID'] == wine_to_grade]
            similarity_score = wine_row['Similarity'].values[0]
            tempScore+=similarity_score    
    
 
    similarity_df.loc[wine_to_grade, 'Accumulated_Similarity'] += tempScore  


num_top_rated_wines = len(high_rated_wines)

similarity_df['Normalized_Similarity'] = similarity_df['Accumulated_Similarity'] / num_top_rated_wines


similarity_df = similarity_df.sort_values(by='Normalized_Similarity', ascending=False)

similarity_df['Normalized_Similarity'] = np.minimum(similarity_df['Normalized_Similarity'], len(high_rated_wines))

display(high_rated_wines[['WineName', 'WineID']].head(10))
display(similarity_df)


,WineName,WineID
190,Pauillac (Grand Cru Classé),111417
193,Pauillac (Grand Cru Classé),111422
195,Saint-Julien (Grand Cru Classé),111431
203,Saint-Estèphe (Grand Cru Classé),111458
209,Sauternes (Premier Grand Cru Classé),111478
400,Brunello di Montalcino,137224
717,Grange,174184
786,Dominus (Christian Moueix),179131
797,Cabernet Sauvignon,179855


,WineName,Accumulated_Similarity,WineID,Normalized_Similarity
Wineid,,,,
183229,Reserve Cabernet Sauvignon,6.306230,183229,0.700692
111417,Pauillac (Grand Cru Classé),6.306230,111417,0.700692
179040,Knights Valley Cabernet Sauvignon,6.306230,179040,0.700692
155334,Camins del Priorat,6.306230,155334,0.700692
155438,Priorat,6.306230,155438,0.700692
...,...,...,...,...
111679,Côtes De Gascogne Classic,0.400000,111679,0.044444
165674,Estelado Brut Rosé,0.400000,165674,0.044444
102392,Terras do Demo Touriga Nacional,0.352419,102392,0.039158


TRYING FOR MULTIPLE USERS (this is using the second evaluation thing)

In [143]:
#TRying evaluation with all top rated wine and 1ìall low rated

fp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, when it shouldn't be similar
fn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, when it should be similar
tp=0 #meaning they are evaluated as similar, and therefore a possible recommendation, and it should be similar
tn=0 #meaning they are evaluated as NOT similar, and therefore NOT a possible recommendation, and it shouldn't be similar
users_ratings = ratings_df.groupby('UserID').count()

selected = users_ratings['Rating'] > 40 #change to 40
selected_users = users_ratings.loc[selected]

print(len(selected_users))

for i in range(len(selected_users)):
    selected_user = selected_users.index[i]
    user_ratings = ratings_df[ratings_df['UserID'] == selected_user].sort_values(by='Rating', ascending=False)
    user_ratings_high = user_ratings[user_ratings['Rating'] >=4.5]
    user_ratings_low = user_ratings[user_ratings['Rating'] < 3]
    high_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_high['WineID'])]
    low_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_low['WineID'])]

    for top_wine in high_rated_wines['WineID'].values:
        similarities = get_similarities(top_wine)  
        for wine_id in high_rated_wines['WineID']: 
            if top_wine == wine_id:
                continue   
            if wine_id in (similarities['WineID'].values):
                wine_row = similarities[similarities['WineID'] == wine_id]
                similarity_score = wine_row['Similarity'].values[0]
                if similarity_score >= 0.8:
                    tp+=1
                if similarity_score <=0.4:
                    fn+=1
        for wine_id in low_rated_wines['WineID']: 
            if wine_id in (similarities['WineID'].values):  
                wine_row = similarities[similarities['WineID'] == wine_id]
                similarity_score = wine_row['Similarity'].values[0]
                if similarity_score <=0.4:
                    tn+=1
                if similarity_score >= 0.8:
                    fp+=1

    for low_wine in low_rated_wines['WineID'].values:
        similarities = get_similarities(low_wine)
        for wine_id in low_rated_wines['WineID']: 
            if top_wine == wine_id:
                continue  
            if wine_id in (similarities['WineID'].values):
                wine_row = similarities[similarities['WineID'] == wine_id]
                similarity_score = wine_row['Similarity'].values[0]
                if similarity_score >= 0.8:
                    tn+=1
                if similarity_score <=0.4:
                    fp+=1
        for wine_id in high_rated_wines['WineID']:   
            if wine_id in (similarities['WineID'].values):
                wine_row = similarities[similarities['WineID'] == wine_id]
                similarity_score = wine_row['Similarity'].values[0]
                if similarity_score <=0.4:
                    tp+=1
                if similarity_score >= 0.8:
                    fn+=1


      
print('tp = '+str(tp))
print('tn = '+str(tn))
print('fp = '+str(fp))
print('fn = '+str(fn))

    

89
tp = 914
tn = 828
fp = 332
fn = 3757


In [128]:
tp = 914
tn = 828
fp = 332
fn = 3757
accuracy = (tn+tp)/(tn+tp+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print ('Accuracy = '+str(accuracy))
print ('Precision = '+str(precision))
print ('Recall = '+str(recall))


Accuracy = 0.2987480706568342
Precision = 0.7335473515248796
Recall = 0.19567544423035751
